# Update embedding vectors for SEU morphologies in Nexus

1. Fetch the embedding resources corresponding to the model id
2. Ask the service[or some python code] for the new embedding vectors
3. Update the embedding resources with the new vectors
4. Update the resources the new model revision in `generation.activity.used.id` (TODO)
4. Update in Nexus
5. Tag them with the model UUID and the its revision (e.g. `e2b953b9-6724-4278-a1e5-3472bd63e374?rev=2`)
6. (TODO) (automatically?) check that the resources are indexed properly

## Setup

In [2]:
import os
import requests
import getpass
import uuid

from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.mappings import DictionaryMapping

from bluegraph.downstream import EmbeddingPipeline
from bluegraph.core import GraphElementEmbedder

In [3]:
TOKEN = getpass.getpass()

········


## Get embedding model

In [4]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN,
                            endpoint = "https://staging.nexus.ocp.bbp.epfl.ch/v1",
                            bucket="dke/embedder_catalog")

In [31]:
MODEL_ID = "https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/dke/embedder_catalog/_/e2b953b9-6724-4278-a1e5-3472bd63e374"
MODEL_REVISION = 9

In [32]:
model_resource = forge.retrieve(f"{MODEL_ID}{'?rev=' + str(MODEL_REVISION) if MODEL_REVISION is not None else ''}")
forge.download(
    model_resource, "distribution.contentUrl", "data", overwrite=True)
pipeline_path = os.path.join("data", model_resource.distribution.name)

In [33]:
pipeline = EmbeddingPipeline.load(
    pipeline_path,
    embedder_interface=GraphElementEmbedder,
    embedder_ext="zip")

In [34]:
embedding_table = pipeline.generate_embedding_table()

## Get existing embedding resources for this model

In [42]:
forge_embeddings = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN, 
                            endpoint="https://staging.nexus.ocp.bbp.epfl.ch/v1",        
                            bucket="dke/seu-embeddings")

In [36]:
existing_resources = forge_embeddings.search(
    {"type": "Embedding",
     "generation": {"activity": {"used": {"id": MODEL_ID}}}},
    limit=None)

In [37]:
for resource in existing_resources:
    morph_id = resource.derivation.entity.id
    resource.embedding = embedding_table.loc[morph_id].tolist()[0].tolist()

__TODO__: Update embedding resources the new model revision in `generation.activity.used.id`

In [38]:
forge_embeddings.update(existing_resources)

<count> 200
<action> _update_many
<succeeded> True


Tag resources with model UUID

In [39]:
tag = f"{MODEL_ID.split('/')[-1]}?rev={MODEL_REVISION}"
forge_embeddings.tag(existing_resources, tag)

<count> 200
<action> _tag_many
<succeeded> True


## Update ES view

In [43]:
forge.sparql(
    """
    https://bbp.epfl.ch/neurosciencegraph/data/views/es/axon-coprojection-embeddings-view
    """)

<action> retrieve
<error> RetrievalError: 404 Client Error: Not Found for url: https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/dke/embedder_catalog/_/https%3A%2F%2Fbbp.epfl.ch%2Fneurosciencegraph%2Fdata%2Fviews%2Fes%2Faxon-coprojection-embeddings-view

